In [28]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [29]:
epl = pd.read_csv("EPL_20_21.csv")
print(len(epl))
#epl.describe()
epl.head()

532


,Name,Club,Nationality,Position,Age,Matches,Starts,Mins,Goals,Assists,Passes_Attempted,Perc_Passes_Completed,Penalty_Goals,Penalty_Attempted,xG,xA,Yellow_Cards,Red_Cards
0,Mason Mount,Chelsea,ENG,"MF,FW",21,36,32,2890,6,5,1881,82.3,1,1,0.21,0.24,2,0
1,Edouard Mendy,Chelsea,SEN,GK,28,31,31,2745,0,0,1007,84.6,0,0,0.00,0.00,2,0
2,Timo Werner,Chelsea,GER,FW,24,35,29,2602,6,8,826,77.2,0,0,0.41,0.21,2,0
3,Ben Chilwell,Chelsea,ENG,DF,23,27,27,2286,3,5,1806,78.6,0,0,0.10,0.11,3,0
4,Reece James,Chelsea,ENG,DF,20,32,25,2373,1,2,1987,85.0,0,0,0.06,0.12,3,0


In [30]:
teams = list(epl["Club"].unique())

matches = []

for i in range(len(teams)):
    for j in range(len(teams)):
        if i != j:
            matches.append([teams[i], teams[j]])

matches = pd.DataFrame(matches, columns=["Home", "Away"])

#epl[epl['Club'] == 'Manchester United']
print(len(matches))

def get_matches(team):
    return np.concatenate([
        np.array(matches[matches["Home"] == team].index),
        np.array(matches[matches["Away"] == team].index)])

team_matches = {t : get_matches(t) for t in teams}


380


In [31]:

def recompute_match_players():

    match_player = np.zeros((len(matches), 2 * len(epl) + 2 + 1 + 3)) # +2 = home, away, +1 won, + 3 odds home draw away

    for i, player in epl.iterrows():
        player_matches = np.random.choice(team_matches[player['Club']], player['Matches'], replace=False)
        match_player[player_matches, i] = 1

        goals = np.random.choice(player_matches, player['Goals'], replace=True)

        for k in goals:
            match_player[k, len(epl) + i] += 1 

    return match_player

match_player = recompute_match_players()

def get_match_id(home, away):
        return np.array(matches[(matches["Home"] == home) & (matches["Away"] == away)].index)[0]

def get_score(home, away):
    mid = get_match_id(home, away)
    home_players = np.array(epl[epl["Club"] == home].index)
    away_players = np.array(epl[epl["Club"] == away].index)

    return (mid, np.sum(match_player[mid, len(epl) + home_players]), np.sum(match_player[mid, len(epl) + away_players]))


print(get_score("Chelsea", "Manchester United"))
print(get_score("Manchester United", "Chelsea"))


(1, 4.0, 4.0)
(38, 4.0, 0.0)


In [32]:
## TODO:
## 1/ iterate and compute the probability of win for each team
## only consider the people that were fielded in the team in the loss function
## meaning that the goals colums[len(players) : 2*len(players)] are input for the NN while the 
## whether the footballer was or not in the team is input for the loss function

## Plot / train calibration!

## 2/ freeze one dataset - considered the true one
## two adversarial updates - one for goals to compute the lambda, one for the odds
## consider initializing the weights to the average for position

In [33]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Input(shape=(len(epl), )),
        #tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(50, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')
    ])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 50)                26650     
                                                                 
 dense_5 (Dense)             (None, 3)                 153       
                                                                 
Total params: 26,803
Trainable params: 26,803
Non-trainable params: 0
_________________________________________________________________


In [34]:
for epoch in range(0, 200):
    print("Epoch", epoch)

    match_player = recompute_match_players()

    for i, r in matches.iterrows():

        ht = r['Home']
        at = r['Away']

        mid, score_home, score_away = get_score(ht, at)

        # lexicographic order
        # always show in the same order the Y column
        if (ht < at):
            first_score = score_home
            second_score = score_away
        else:
            second_score = score_home
            first_score = score_away

        match_player[mid, 2 * len(epl)] = first_score
        match_player[mid, 2 * len(epl) + 1] = second_score
        match_player[mid, 2 * len(epl) + 2] = int(first_score > second_score)
        match_player[mid, 2 * len(epl) + 3] = int(first_score == second_score)
        match_player[mid, 2 * len(epl) + 4] = int(first_score < second_score)

    idxs = list(range(0, len(epl))) # players
    start_label = 2 * len(epl) + 2
    idxs += [start_label, start_label + 1, start_label + 2] # score

    dataset = tf.data.Dataset.from_tensor_slices(match_player[:, idxs]).map(lambda x: (x[:-3], x[-3:])).shuffle(1024).batch(64)

    #for k, v in dataset.take(1):
    #    print(k, v)

    model.fit(dataset, epochs=1, verbose=2)



Epoch 0
6/6 - 0s - loss: 1.0889 - accuracy: 0.3921 - 221ms/epoch - 37ms/step


2022-09-16 20:09:16.887769: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 1
6/6 - 0s - loss: 1.0908 - accuracy: 0.4000 - 56ms/epoch - 9ms/step
Epoch 2
6/6 - 0s - loss: 1.0680 - accuracy: 0.3895 - 51ms/epoch - 8ms/step
Epoch 3
6/6 - 0s - loss: 1.1123 - accuracy: 0.4289 - 49ms/epoch - 8ms/step
Epoch 4
6/6 - 0s - loss: 1.1028 - accuracy: 0.3974 - 52ms/epoch - 9ms/step
Epoch 5
6/6 - 0s - loss: 1.0776 - accuracy: 0.4289 - 50ms/epoch - 8ms/step
Epoch 6
6/6 - 0s - loss: 1.0829 - accuracy: 0.3947 - 60ms/epoch - 10ms/step
Epoch 7
6/6 - 0s - loss: 1.0645 - accuracy: 0.4237 - 53ms/epoch - 9ms/step
Epoch 8
6/6 - 0s - loss: 1.0722 - accuracy: 0.4474 - 52ms/epoch - 9ms/step
Epoch 9
6/6 - 0s - loss: 1.0839 - accuracy: 0.4053 - 53ms/epoch - 9ms/step
Epoch 10
6/6 - 0s - loss: 1.0902 - accuracy: 0.4053 - 50ms/epoch - 8ms/step
Epoch 11
6/6 - 0s - loss: 1.0619 - accuracy: 0.4342 - 50ms/epoch - 8ms/step
Epoch 12
6/6 - 0s - loss: 1.0935 - accuracy: 0.4105 - 52ms/epoch - 9ms/step
Epoch 13
6/6 - 0s - loss: 1.0852 - accuracy: 0.4105 - 50ms/epoch - 8ms/step
Epoch 14
6/6 - 0s - 

In [35]:
v = np.empty([1, len(epl)])
match_player = recompute_match_players()


def predict(h, a):
    mid = get_match_id(h, a)
    v[0] = match_player[mid, :len(epl)]
    p = model.predict(v)

    mid, home, away = get_score(h, a)

    if (h < a):
        print(h, a, home, away, 1/p)
    else:
        print(a, h, away, home, 1/p)


predict("Manchester City", "Chelsea")
predict("Chelsea", "Manchester City")
predict("Manchester City", "Southampton")
predict("Southampton", "Manchester City")
predict("Southampton", "Chelsea")
predict("Chelsea", "Southampton")

Chelsea Manchester City 1.0 1.0 [[4.536013  5.939192  1.6362087]]
Chelsea Manchester City 2.0 1.0 [[4.5628724 5.326914  1.6860168]]
Manchester City Southampton 3.0 0.0 [[1.4948896 6.1018815 5.9819202]]
Manchester City Southampton 2.0 0.0 [[2.1928337 4.4607944 3.1270163]]
Chelsea Southampton 0.0 3.0 [[2.5810845 3.5542815 3.0191855]]
Chelsea Southampton 0.0 0.0 [[2.3024163 3.8449957 3.2723005]]


2022-09-16 20:10:02.493391: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
